# Applied Data Science Capstone - Cousera
---

## _(this notebook will be used specifically for capstone project)_
***

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course")

Hello Capstone Project Course


## Segmenting and Clustering Neighborhoods in Toronto - _Part - 2_
---

In [7]:
#!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    beautifulsoup4-4.9.0       |   py36h9f0ad1d_0         160 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m       conda-forge

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                

In [1]:
#!conda install -c conda-forge lxml --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    lxml-4.4.1                 |   py36h7ec2d77_0         1.6 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m          conda-forge

The following packages will be UPDATED:

    ca-certificates: 2020.1.1-0                       --

### Create BeautifulSoup object for Scraping Webpage

In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)

soup=BeautifulSoup(r.text,'lxml')

### Scrape the table from Wiki

In [4]:
match=soup.find('table',class_='wikitable sortable')

A=[]
B=[]
C=[]
rows = match.find_all('tr')
for row in rows:
    cols=row.find_all('td')
    for i,x in enumerate(cols):
            if i==0:
                A.append(x.text.strip())
            elif i==1:
                B.append(x.text.strip())
            else:
                C.append(x.text.strip())
    

### Creating a Dataframe from the scraped data

In [5]:
#Create a distionary
Dict={'Postal':A,'Borough':B,'Neighborhood':C}

#create a panda Dataframe
DF=pd.DataFrame(Dict)
#remove borough with value 'Not assigned'
DF=DF[DF['Borough']!='Not assigned'].reset_index(drop=True)
#replace '/' with ',' in the neighborhood string
DF['Neighborhood']=DF['Neighborhood'].replace(" / ",",",regex=True)

In [6]:
DF.shape

(103, 3)

### Get the latitude and the longitude coordinates of each neighborhood

In [7]:
#import the Geolocation Data
loc=pd.read_csv('http://cocl.us/Geospatial_data')
#Merge the Neighborhood data with the Geo-location Data
Df_loc=pd.merge(DF,loc,left_on='Postal',right_on='Postal Code').drop('Postal Code',axis=1)

### Explore and cluster the neighborhoods in Toronto

#### Define Foursquare credentials

In [8]:
CLIENT_ID = 'NDCW0CMHZ32LOVQZULUCVGBEKFVL1C4BMOUX2EA30VNGAKZF' # your Foursquare ID
CLIENT_SECRET = 'I3PMUWTGP5FBBR3VUYJJESOMPHCEO5UK4INUMZCLQMJJWYQX' # your Foursquare Secret
VERSION = '20200504' # Foursquare API version
LIMIT=100 #Set Limit
radius=500 #set radius of search

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
toranto_venues = getNearbyVenues(names=Df_loc['Neighborhood'],
                                   latitudes=Df_loc['Latitude'],
                                   longitudes=Df_loc['Longitude']
                                  )



Parkwoods
Victoria Village
Regent Park,Harbourfront
Lawrence Manor,Lawrence Heights
Queen's Park,Ontario Provincial Government
Islington Avenue
Malvern,Rouge
Don Mills
Parkview Hill,Woodbine Gardens
Garden District,Ryerson
Glencairn
West Deane Park,Princess Gardens,Martin Grove,Islington,Cloverdale
Rouge Hill,Port Union,Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate,Bloordale Gardens,Old Burnhamthorpe,Markland Wood
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Wilson Heights,Downsview North
Thorncliffe Park
Richmond,Adelaide,King
Dufferin,Dovercourt Village
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Union Station,Toronto Islands
Little Portugal,Trinity
Kennedy Park,Ionview,East Birchmount Park
Bayview Village
Downsview
The Danforth West,Riverdale
Toronto Dominion Centre

### Let's check the size of the resulting dataframe

In [33]:
toranto_venues.shape

(2142, 7)

### Check if all Neighborhoods have 10 Venue Locations
### Remove Neighborhoods with less than 10 Venue Locations

In [34]:
Venue_list=toranto_venues[['Neighborhood','Venue']].groupby('Neighborhood').count()
No_ten_Venue=Venue_list[Venue_list['Venue']<10]
Venue_list=Venue_list[Venue_list['Venue']>=10]
print('here are the list of neighborhoods with less than 10 Venues: ',No_ten_Venue)

here are the list of neighborhoods with less than 10 Venues:                                                      Venue
Neighborhood                                             
Agincourt                                               5
Alderwood,Long Branch                                   9
Bayview Village                                         4
Birch Cliff,Cliffside West                              4
Caledonia-Fairbanks                                     4
Cedarbrae                                               8
Cliffside,Cliffcrest,Scarborough Village West           3
Davisville North                                        7
Del Ray,Mount Dennis,Keelsdale and Silverthorn          6
Dorset Park,Wexford Heights,Scarborough Town Ce...      7
East Toronto                                            4
Eringate,Bloordale Gardens,Old Burnhamthorpe,Ma...      7
Forest Hill North & West                                4
Glencairn                                               6
Guildwood,

In [35]:
toranto_venues=toranto_venues[toranto_venues['Neighborhood'].isin(Venue_list.index)]
toranto_venues.shape


(1946, 7)

### Analyze Each Neighborhood

In [36]:
# one hot encoding
toranto_onehot = pd.get_dummies(toranto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toranto_onehot['Neighborhood'] = toranto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toranto_onehot.columns[-1]] + list(toranto_onehot.columns[:-1])
toranto_onehot = toranto_onehot[fixed_columns]

### Let's confirm the new size

In [37]:
toranto_grouped = toranto_onehot.groupby('Neighborhood').mean().reset_index()
toranto_grouped.shape

(49, 249)

### Let's print each neighborhood along with the top 10 most common venues

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toranto_grouped['Neighborhood']

for ind in np.arange(toranto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toranto_grouped.iloc[ind, :], num_top_venues)

In [40]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Wilson Heights,Downsview North",Coffee Shop,Bank,Shopping Mall,Ice Cream Shop,Sushi Restaurant,Middle Eastern Restaurant,Restaurant,Bridal Shop,Deli / Bodega,Fried Chicken Joint
1,"Bedford Park,Lawrence Manor East",Coffee Shop,Sushi Restaurant,Restaurant,Sandwich Place,Italian Restaurant,Comfort Food Restaurant,Pharmacy,Pizza Place,Café,Butcher
2,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Bakery,Restaurant,Cheese Shop,Café,Breakfast Spot,Jazz Club
3,"Brockton,Parkdale Village,Exhibition Place",Café,Nightclub,Coffee Shop,Breakfast Spot,Performing Arts Venue,Stadium,Restaurant,Bakery,Intersection,Italian Restaurant
4,Business reply mail Processing Centre,Light Rail Station,Pizza Place,Burrito Place,Fast Food Restaurant,Skate Park,Auto Workshop,Farmers Market,Spa,Brewery,Comic Shop


### Cluster Neighborhoods

In [41]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toranto_grouped_clustering = toranto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toranto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 3, 0], dtype=int32)

### Remove neighborhoods with less than 10 venues from the Initial list

In [56]:
Df_loc_new=Df_loc[Df_loc['Neighborhood'].isin(Venue_list.index)].reset_index(drop=True)

In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toranto_merged = Df_loc_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toranto_merged = toranto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toranto_merged.head() # check the last columns!

,Postal,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Farmers Market,Event Space
1,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Women's Store,Furniture / Home Store,Accessories Store,Gift Shop,Vietnamese Restaurant,Coffee Shop,Event Space,Miscellaneous Shop,Boutique
2,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Bank,Bar,Burrito Place,Burger Joint,Beer Bar,Japanese Restaurant
3,M3B,North York,Don Mills,43.745906,-79.352188,0,Restaurant,Asian Restaurant,Coffee Shop,Gym,Japanese Restaurant,Beer Store,Discount Store,Sporting Goods Shop,Italian Restaurant,Sandwich Place
4,M4B,East York,"Parkview Hill,Woodbine Gardens",43.706397,-79.309937,2,Pizza Place,Pet Store,Bank,Breakfast Spot,Fast Food Restaurant,Intersection,Athletics & Sports,Pharmacy,Gym / Fitness Center,Gastropub


In [55]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.21.0-py_0 conda-forge


geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [44]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [67]:
# create map
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
#lat,long of torans ato
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#create a folium chart
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=10)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toranto_merged['Latitude'], toranto_merged['Longitude'], toranto_merged['Neighborhood'], toranto_merged['Cluster Labels'].astype('int')):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining Cluster 1

In [72]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 0, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Restaurant,Café,Theater,Farmers Market,Event Space
2,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Italian Restaurant,Café,Bank,Bar,Burrito Place,Burger Joint,Beer Bar,Japanese Restaurant
3,North York,0,Restaurant,Asian Restaurant,Coffee Shop,Gym,Japanese Restaurant,Beer Store,Discount Store,Sporting Goods Shop,Italian Restaurant,Sandwich Place
5,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Restaurant,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Lingerie Store
6,North York,0,Restaurant,Asian Restaurant,Coffee Shop,Gym,Japanese Restaurant,Beer Store,Discount Store,Sporting Goods Shop,Italian Restaurant,Sandwich Place
7,East York,0,Park,Cosmetics Shop,Skating Rink,Beer Store,Diner,Curling Ice,Athletics & Sports,Asian Restaurant,Pharmacy,Video Store
8,Downtown Toronto,0,Café,Coffee Shop,American Restaurant,Gastropub,Cocktail Bar,Restaurant,Italian Restaurant,Gym,Beer Bar,Department Store
9,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Beer Bar,Bakery,Restaurant,Cheese Shop,Café,Breakfast Spot,Jazz Club
10,East York,0,Coffee Shop,Sporting Goods Shop,Bank,Furniture / Home Store,Burger Joint,Supermarket,Electronics Store,Department Store,Fish & Chips Shop,Bagel Shop
11,Downtown Toronto,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Thai Restaurant,Salad Place,Restaurant,Bubble Tea Shop,Burger Joint


### Examining Cluster 2

In [73]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 1, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Clothing Store,Women's Store,Furniture / Home Store,Accessories Store,Gift Shop,Vietnamese Restaurant,Coffee Shop,Event Space,Miscellaneous Shop,Boutique
17,North York,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Women's Store,Toy / Game Store,Bakery,Electronics Store,Japanese Restaurant,Tea Room
35,Central Toronto,1,Clothing Store,Coffee Shop,Bagel Shop,Park,Chinese Restaurant,Café,Mexican Restaurant,Rental Car Location,Restaurant,Salon / Barbershop


### Examining Cluster 3

In [74]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 2, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East York,2,Pizza Place,Pet Store,Bank,Breakfast Spot,Fast Food Restaurant,Intersection,Athletics & Sports,Pharmacy,Gym / Fitness Center,Gastropub
42,Scarborough,2,Pharmacy,Pizza Place,Coffee Shop,Noodle House,Bank,Italian Restaurant,Fast Food Restaurant,Thai Restaurant,Fried Chicken Joint,Chinese Restaurant
47,Scarborough,2,Chinese Restaurant,Fast Food Restaurant,Breakfast Spot,Supermarket,Bank,Sandwich Place,Nail Salon,Coffee Shop,Pizza Place,Pharmacy


### Examining Cluster 4

In [75]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 3, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,3,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Candy Store,Restaurant,Diner,Italian Restaurant,Baby Store
20,North York,3,Grocery Store,Park,Liquor Store,Food Truck,Hotel,Athletics & Sports,Baseball Field,Discount Store,Bank,Gym / Fitness Center
25,North York,3,Grocery Store,Park,Liquor Store,Food Truck,Hotel,Athletics & Sports,Baseball Field,Discount Store,Bank,Gym / Fitness Center
28,North York,3,Grocery Store,Park,Liquor Store,Food Truck,Hotel,Athletics & Sports,Baseball Field,Discount Store,Bank,Gym / Fitness Center
32,North York,3,Grocery Store,Park,Liquor Store,Food Truck,Hotel,Athletics & Sports,Baseball Field,Discount Store,Bank,Gym / Fitness Center


### Examining Cluster 5

In [76]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 4, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Scarborough,4,Bus Line,Bakery,Bus Station,Park,Metro Station,Intersection,Ice Cream Shop,Soccer Field,Convenience Store,Comfort Food Restaurant
